In [1]:
import os
import time
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.cluster import KMeans
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import datetime
from datetime import datetime as dt

import random

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
filepath = './horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC).mp4'
filepath_check = './checkpoint/'
input_dim = 500
maxCorners = 100
frames = 30
net_len = 100
units = 100
output_size = 2
classes_list = ["stop", "go"]

In [3]:
horizontal = ['./horizontal/Video-7-36-30-868-22-03-2022/Video-7-36-30-868 (S1WK)',
            #'./horizontal/Video-10-6-45-47-30-03-2022/Video-10-6-45-47 (J57V)',
            './horizontal/Video-15-43-13-884-22-03-2022/Video-15-43-13-884 (J4NP)',
            './horizontal/Video-10-25-45-905-31-03-2022/Video-10-25-45-905 (3W3K)',
            './horizontal/Video-10-56-13-801-31-03-2022/Video-10-56-13-801 (X0N9)',            
            './horizontal/Video-11-19-18-132-30-03-2022/Video-11-19-18-132 (SH2G)',
            './horizontal/Video-12-51-53-673-27-03-2021/Video-12-51-53-673 (NKXJ)',
            './horizontal/Video-15-26-20-118-29-03-2022/Video-15-26-20-118 (5FLZ)',
            './horizontal/Video-15-32-33-709-31-03-2022/Video-15-32-33-709 (7BAA)',
            './horizontal/Video-15-41-30-94-29-03-2022/Video-15-41-30-94 (PH33)',
            #'./horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC)',
            './horizontal/Video-15-49-30-274-29-03-2022/Video-15-49-30-274 (JP92)',
            './horizontal/Video-15-8-58-789-27-03-2022/Video-15-8-58-789 (SSQM)',
            './horizontal/Video-17-14-15-118-06-03-2021/Video-17-14-15-118 (PE9P)',
            './horizontal/Video-17-34-45-652-25-03-2021/Video-17-34-45-652 (BK89)',
            './horizontal/Video-17-44-40-74-06-03-2021/Video-17-44-40-74 (CLNF)',
            './horizontal/Video-7-51-44-763-22-03-2022/Video-7-51-44-763 (1QN6)',
            './horizontal/Video-8-6-54-707-22-03-2022/Video-8-6-54-707 (PKAY)']

vertical = ['./vertical/Video-10-15-12-812-14-11-2020/Video-10-15-12-812 (2Y5M)',
           './vertical/Video-10-30-21-575-29-09-2021/Video-10-30-21-575 (UOM8)',
           './vertical/Video-11-21-2-419-13-09-2021/Video-11-21-2-419 (KVYG)',
           './vertical/Video-11-21-6-412-28-12-2020/Video-11-21-6-412 (JQQM)',
           './vertical/Video-11-36-12-363-28-09-2021/Video-11-36-12-363 (8119)',
           './vertical/Video-11-36-16-421-28-12-2020/Video-11-36-16-421 (PFAQ)',
           './vertical/Video-11-5-49-336-13-09-2021/Video-11-5-49-336 (CZPL)',
           './vertical/Video-12-23-51-640-02-10-2021/Video-12-23-51-640 (WLWK)',
           './vertical/Video-12-39-1-595-02-10-2021/Video-12-39-1-595 (QDUT)',
           './vertical/Video-12-44-36-682-24-09-2021/Video-12-44-36-682 (ATO6)',
           './vertical/Video-13-16-32-822-10-09-2021/Video-13-16-32-822 (UQ1Q)',
           './vertical/Video-13-20-30-695-15-11-2021/Video-13-20-30-695 (VLFO)',
           './vertical/Video-13-31-46-773-10-09-2021/Video-13-31-46-773 (MXM7)',
           './vertical/Video-16-16-44-458-17-11-2020/Video-16-16-44-458 (LCRL)',
           './vertical/Video-16-36-42-174-10-11-2020/Video-16-36-42-174 (IXTE)',
           './vertical/Video-16-47-10-870-17-11-2020/Video-16-47-10-870 (YFCN)',
           './vertical/Video-17-17-30-919-17-11-2020/Video-17-17-30-919 (ID6X)',
           './vertical/Video-17-32-40-752-17-11-2020/Video-17-32-40-752 (E575)',
           './vertical/Video-17-47-40-713-02-10-2021/Video-17-47-40-713 (E7IK)',
           './vertical/Video-20-21-59-727-24-09-2021/Video-20-21-59-727 (S04R)',
           './vertical/Video-20-45-56-152-24-10-2021/Video-20-45-56-152 (6M38)']

random.shuffle(horizontal)
random.shuffle(vertical)

videopath = horizontal + vertical

random.shuffle(videopath)

In [4]:
def six_or_else(a):
    if a[1]==6:
        return [1,a[2]]
    else:
        return [0,a[2]]

def mode_periods(df):
    last_mode = -1
    mode_time = []
    mode_time.append([1 if df.iloc[0][0] == 6 else 0 , '00:00:00.0000000']) 
    for row in df.itertuples():  
        if row[1] != last_mode:   
            mode_time.append(six_or_else(row))
        last_mode = row[1]
    mode_time.append([1 if df.iloc[-1][0] == 6 else 0 , '1'+df.iloc[-1][1][1:] ] ) 
    
    return mode_time

In [5]:
def get_RNN(units, input_dim, output_size):
    model = keras.Sequential()
    model.add(layers.LSTM(units, return_sequences=True, input_shape=(None, 2)))
    model.add(layers.GRU(256))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(output_size))

    return model

In [16]:
def data(filepath):
    df = pd.read_csv(filepath[:-4]+'.csv')[['activityMode','timestamp']]
    mode_time = mode_periods(df)
    
    # Read the video 
    cap = cv2.VideoCapture(filepath)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners = maxCorners,
                       qualityLevel = 0.07,
                       minDistance = 10,
                       blockSize = 5 )

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    net = []
    label_list = []
    i_frame = 0
    j_frame = 0
    while True:       
        ret, old_frame = cap.read()
        if not ret:
                break
        old_frame = cv2.resize(old_frame, (input_dim, input_dim))
        height, width, _ = old_frame.shape 
        part_width = round(width * 0.4)
        part_height = round(height * 0.4)
        
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        cv2.rectangle(old_gray, (part_width, part_height), 
                          (width - part_width, height - part_height), 0, -1)
        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_frame)
        
        while True:
            # Read new frame
            ret, frame = cap.read()
            
            if not ret:
                break
                
            sec = cap.get(cv2.CAP_PROP_POS_MSEC)
            res = datetime.timedelta(seconds = sec/1000)
            for i in range(len(mode_time)-1): 
                if (dt.strptime(mode_time[i][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) <= res) and\
                    (dt.strptime(mode_time[i+1][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) >= res):
                    label = [mode_time[i][0]]                
                    break  
                    
            if (len(label_list)!=0)and(label == label_list[-1])and(j_frame>maxCorners):
                continue 
                
            frame = cv2.resize(frame, (input_dim, input_dim))
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.rectangle(old_gray, (part_width, part_height), 
                          (width - part_width, height - part_height), 0, -1)
            # Calculate Optical Flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params
            )

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            
            new_net = good_new - good_old
            #print(new_net)
            while new_net.shape[0] < maxCorners:                
                new_net = np.append(new_net, [[0,0]], axis=0)                  
            new_net = np.array(new_net)
            new_net[np.lexsort((new_net[:,1],new_net[:,0]))]
            
            i_frame += 1
            
            if True:#(len(label_list)==0):
                j_frame += 1
                net.append(new_net) 
                label_list.append(label)
                # Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
                
                #if j_frame >= 100:
                   # break
                continue
            
            if ((label == label_list[-1]) and (j_frame<=frames)):
                j_frame += 1
                net.append(new_net) 
                label_list.append(label)
                # Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
                if j_frame >= frames:
                    break
                continue           
                        
            if (label != label_list[-1]):
                j_frame = 0
                net.append(new_net) 
                label_list.append(label)
                # Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
                break
                
    return np.array(net)/input_dim, np.array(label_list)

In [14]:
def lucas_kanade_method_orig(video_path, callbacks, val_x, val_y):
    df = pd.read_csv(video_path[:-4]+'.csv')[['activityMode','timestamp']]
    mode_time = mode_periods(df)
    
    model = get_RNN(units, input_dim, output_size)
    
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer="sgd",
        metrics=["accuracy"]
    )
    
    # Read the video 
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners = maxCorners,
                       qualityLevel = 0.07,
                       minDistance = 7,
                       blockSize = 7 )

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(winSize  = (15, 15),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create random colors
    color = np.random.randint(0, 255, (500, 3))
    predicted_class_name = ' '
    while True:       
        start = time.time()
        # Take first frame and find corners in it
        ret, old_frame = cap.read()
        old_frame = cv2.resize(old_frame, (input_dim, input_dim))
        height, width, _ = old_frame.shape 
        part_width = round(width * 0.3)
        part_height = round(height * 0.3)
        
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        
        cv2.rectangle(old_gray, (part_width, part_height), 
                          (width - part_width, height - part_height), 0, -1)
        
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_frame)
        net = []
        label_list = []
        
        while True:
            # Read new frame
            ret, frame = cap.read()
            frame = cv2.resize(frame, (input_dim, input_dim))
            if not ret:
                break

            cv2.rectangle(frame, (part_width, part_height), 
                          (width - part_width, height - part_height), 0, -1)
            
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Calculate Optical Flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params
            )

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            
            new_net = good_new - good_old
            #print(new_net)
            while new_net.shape[0] < maxCorners:                
                new_net = np.append(new_net, [[0,0]], axis=0)                  
            new_net = np.array(new_net)
            new_net[np.lexsort((new_net[:,1],new_net[:,0]))]   
            net.append(new_net)
            
            # Draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

            sec = cap.get(cv2.CAP_PROP_POS_MSEC)
            res = datetime.timedelta(seconds = sec/1000)
            for i in range(len(mode_time)-1): 
                if (dt.strptime(mode_time[i][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) <= res) and\
                    (dt.strptime(mode_time[i+1][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) >= res):
                    label = [mode_time[i][0]]                
                    break  
            
            label_list.append(label)                
                
            # Display the demo
            img = cv2.add(frame, mask)
            cv2.putText(img, predicted_class_name, 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("frame", img)
            k = cv2.waitKey(25) & 0xFF
            if k == 27:
                break

            # Update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)
            if np.array(net).shape[0] >= frames:
                net = np.array(net) / input_dim
                #history = model.fit(
                    #net, np.array(label_list), batch_size=20, epochs=1,
                    #callbacks=callbacks,
                    #validation_data  = (val_x, val_y) 
                #)
                predicted_labels_probabilities = model.predict(net)[0]
                predicted_label = np.argmax(predicted_labels_probabilities)
        
                # Accessing The Class Name using predicted label.
                predicted_class_name = classes_list[predicted_label]
                
                break

In [9]:
val_x, val_y = data('./horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC).mp4')
print(val_x.shape, val_y.shape)

(101, 100, 2) (101, 1)


In [10]:
checkpoint = keras.callbacks.ModelCheckpoint(
        filepath_check, monitor="val_accuracy",
        save_best_only=True, verbose=1,
        mode='max')
callbacks = [checkpoint]

In [18]:
# Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15,
                                        mode = 'max', restore_best_weights = True)

model = get_RNN(units, input_dim, output_size)

model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer="sgd",
        metrics=["accuracy"]
    )

for i in videopath:    
    df = pd.read_csv(i+'.csv')[['activityMode','timestamp']]
    mode_time = mode_periods(df)
    
    #if len(mode_time)<=4:
        #continue
    
    x, y = data(i + '.mp4')   
      
    model.fit(
            np.array(x), np.array(y), batch_size=20, epochs=3,
            callbacks=[early_stopping_callback],
            validation_data  = (val_x, val_y) 
        )

Epoch 1/3


KeyboardInterrupt: 

In [19]:
model.fit(
            val_x, val_y, batch_size=20, epochs=15,
            callbacks=[early_stopping_callback],
            validation_data  = (val_x, val_y)
        )

Epoch 1/15
6/6 [==============================] - 3s 466ms/step - loss: 0.6859 - accuracy: 0.7723 - val_loss: 0.6452 - val_accuracy: 1.0000
Epoch 2/15
6/6 [==============================] - 1s 192ms/step - loss: 0.6229 - accuracy: 1.0000 - val_loss: 0.5739 - val_accuracy: 1.0000
Epoch 3/15
6/6 [==============================] - 1s 174ms/step - loss: 0.5505 - accuracy: 1.0000 - val_loss: 0.4976 - val_accuracy: 1.0000
Epoch 4/15
6/6 [==============================] - 1s 168ms/step - loss: 0.4767 - accuracy: 1.0000 - val_loss: 0.4280 - val_accuracy: 1.0000
Epoch 5/15
6/6 [==============================] - 1s 162ms/step - loss: 0.4099 - accuracy: 1.0000 - val_loss: 0.3688 - val_accuracy: 1.0000
Epoch 6/15
6/6 [==============================] - 1s 173ms/step - loss: 0.3527 - accuracy: 1.0000 - val_loss: 0.3173 - val_accuracy: 1.0000
Epoch 7/15
6/6 [==============================] - 1s 155ms/step - loss: 0.3063 - accuracy: 1.0000 - val_loss: 0.2740 - val_accuracy: 1.0000
Epoch 8/15
6/6 [====

In [20]:
lucas_kanade_method_orig('./horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC).mp4',
                         callbacks, val_x, val_y)

KeyboardInterrupt: 